#### Install the useful packages.

In [4]:
myPackages <- c("easypackages")
#the argument `lib.loc = "~/rlib"` is used because the packages were installed in this folder in my case
lapply(myPackages, library, lib.loc = "~/rlib", character.only = TRUE) 
libraries("dplyr", "tidyr")

[[1]]
[1] "easypackages" "stats"        "graphics"     "grDevices"    "utils"       
[6] "datasets"     "methods"      "base"

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr

All packages loaded successfully



#### Import the gallup data that contain every country.

In [13]:
gallupRaw <- rio::import("~/gallup data/The_Gallup_042722.dat")

Ambiguous file format ('.dat'), but attempting 'data.table::fread("~/gallup data/The_Gallup_042722.dat")'



We need country name, country code, weight, survey date, survey year, ls, hope, PA, NA

Filter the data and save the it for future use.

In [14]:
gallupExtracted <- gallupRaw %>% select(COUNTRYNEW, COUNTRY_ISO3, WGT, 
                                        FIELD_DATE, WP4.INTERVIEWDATE = WP4, YEAR_WAVE, 
                                        YEAR_CALENDAR, ls = WP16, hope = WP18, 
                                        smile = WP63, enjoy = WP67, worry = WP69, 
                                        sad = WP70, stress = WP71, anger = WP74)
write.csv(gallupExtracted, "gallupExtracted.csv", row.names = FALSE)

In [15]:
gallupExtracted <- rio::import("gallupExtracted.csv")
colnames(gallupExtracted)
dim(gallupExtracted)

[1] "COUNTRYNEW"        "COUNTRY_ISO3"      "WGT"              
 [4] "FIELD_DATE"        "WP4.INTERVIEWDATE" "YEAR_WAVE"        
 [7] "YEAR_CALENDAR"     "ls"                "hope"             
[10] "smile"             "enjoy"             "worry"            
[13] "sad"               "stress"            "anger"

[1] 2450403      15

Create a function for changing the "DK" / "Refused" response to NA.

In [16]:
na_codes <- function(x, ...) {
    for(y in list(...))
        x[x == y] <- NA
    x
}

Change the "DK" / "Refused" response to NA

Change the responses to SWB from [1 Yes, 2 No] to [1 Yes, 0 No] for getting the means for individual positive affect and negative affect

In [ ]:
gallupAssas <- gallupExtracted
#convert empty WP4.INTERVIEWDATE to NA
gallupAssas <- gallupAssas %>% mutate(WP4.INTERVIEWDATE = na_if(WP4.INTERVIEWDATE, "")) 
#life satisfication
gallupAssas$ls <- na_codes(gallupAssas$ls, 98, 99)
#hope
gallupAssas$hope <- na_codes(gallupAssas$hope, 98, 99)
#pa and #na
gallupAssas$smile <- 2 - na_codes(gallupAssas$smile, 3, 4)
gallupAssas$enjoy <- 2 - na_codes(gallupAssas$enjoy, 3, 4)
gallupAssas$worry <- 2 - na_codes(gallupAssas$worry, 3, 4)
gallupAssas$sad <- 2 - na_codes(gallupAssas$sad, 3, 4)
gallupAssas$stress <- 2 - na_codes(gallupAssas$stress, 3, 4)
gallupAssas$anger <- 2 - na_codes(gallupAssas$anger, 3, 4)
gallupAssas <- gallupAssas %>% 
 rowwise() %>% 
 mutate(pa = mean(cbind(enjoy, smile), na.rm=TRUE),
        na = mean(cbind(worry, sad, stress, anger), na.rm=TRUE)) %>% ungroup
head(gallupAssas)

In [ ]:
write.csv(gallupAssas, "gallupAssas.csv")

In [ ]:
colnames(gallupAssas)
dim(gallupAssas)